## Calculating theoretical tri-palmitin yield from glucose using a yeast metabolic model

This [Jupyter notebook](https://jupyter.org/) describe the use of a genome scale metabolic model to predict the maximum
theoretical yield tripalmitin of a hypothetical yeast strain that is engineered to export this product.

The [cobrapy](https://github.com/opencobra/cobrapy) software package was used to perform the calculations. The motivation 
was to learn how to use cobrapy.

During the writing of this notebook I took aprt in this exchange on the [cobrapy google group](https://groups.google.com/forum/#!topic/cobra-pie/-sPHjNsB974). I am grateful for all the useful input I got.

## Definition of the objective

The objective is optimizing for the production of tripalmitin (C51H98O6 MW 807.32 g/mol) from D-glucose under non-limiting conditions of oxygen.

![Tripalmitin](trip.jpg "The structure of Tripalmitin")

According to [Nelson DL, Cox M. Lehninger Principles of Biochemistry: 7th Int Ed 2017](https://www.macmillanlearning.com/Catalog/product/lehningerprinciplesofbiochemistry-seventhedition-nelson) tri-acyl glycerols are made from Phosphatidic acid (FIGURE 21-17).

![FIGURE 21-17](21-17.png "Biosynthesis of phosphatidic acid")

**FIGURE 21-17** Biosynthesis of phosphatidic acid. A fatty acyl group is
activated by formation of the fatty acyl–CoA, then transferred to ester linkage
with L -glycerol 3-phosphate, formed in either of the two ways shown.
Phosphatidic acid is shown here with the correct stereochemistry ( L ) at C-2 of
the glycerol molecule. (The intermediate product with only one esterified fatty
acyl group is lysophosphatidic acid.) To conserve space in subsequent figures
(and in Fig. 21-14), both fatty acyl groups of glycerophospholipids, and all
three acyl groups of triacylglycerols, are shown projecting to the right.


Phosphatidic acid is used for tri-acyl glycerol synthesis.


![FIGURE 21-18](21-18.png "Phosphatidic acid in lipid biosynthesis")

**FIGURE 21-18** Phosphatidic acid in lipid biosynthesis. Phosphatidic acid is
the precursor of both triacylglycerols and glycerophospholipids. The
mechanisms for head-group attachment in phospholipid synthesis are described
later in this section.

The overall reaction is the following:

    Glycerol 3-phosphate + 3 Palmitoyl-CoA + 2 ATP -> tripalmitin + 3 Phosphate + 2 ADP + 3 CoA

I will further assume that the reaction will occur in the cytosol.


## Metabolic model

I will use the [iMM904](http://bigg.ucsd.edu/models/iMM904) model. Many models can be found in the [BiGG](http://bigg.ucsd.edu/) model repository
and cobrapy can automatically load these models. The model itself was described by [Mo et al. 2009](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2679711/) and according to the publication include 904 genes and 1,412 reactions. The BiGG entry for this model lists a higher number of genes and reactions, so the model has apparently been updated.

There are other models like the [yeast-GEM](https://github.com/SysBioChalmers/yeast-GEM) that has more genes and reactions.

In [1]:
import cobra

I chose to download the model, I opted to the JSON version. There are also other versions available from the BiGG site.
The *load_json_model* can be used to read the model from the local JSON file.

In [2]:
iMM904 = cobra.io.load_json_model("iMM904.json")

Since I will modifiy the modle I make a copy so that I have the original if I need it.

In [3]:
model = iMM904.copy()

In [4]:
model

Name,iMM904
Memory address,0x07f3e7c6dd0f0
Number of metabolites,1226
Number of reactions,1577
Number of groups,0
Objective expression,1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, mitochondria, extracellular space, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, nucleus, golgi apparatus"


The model comes with a set of constraints called *medium*. 
This can be thought of as contraints posed by the composition of the 
extracellular medium or by the cultivation regiment.

We can see below that external oxygen exchange
EX_o2_e is limited to 2.0 while others are set to 999999.

I discovered this when I got ethanol production in my initial attempts.

In [5]:
model.medium

{'EX_fe2_e': 999999.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_na1_e': 999999.0,
 'EX_nh4_e': 999999.0,
 'EX_o2_e': 2.0,
 'EX_pi_e': 999999.0,
 'EX_so4_e': 999999.0}

I found the code below [here](https://cobrapy.readthedocs.io/en/latest/media.html) used to
change the medium. 

The code seem a bit convoluted, but seetting it directly does not work.

In [6]:
medium = model.medium
medium["EX_o2_e"] = 999999.0
model.medium = medium
model.medium

{'EX_fe2_e': 999999.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_na1_e': 999999.0,
 'EX_nh4_e': 999999.0,
 'EX_o2_e': 999999.0,
 'EX_pi_e': 999999.0,
 'EX_so4_e': 999999.0}

The raction has the following metabolites:

- Glycerol 3-phosphate
- Palmitoyl-CoA
- ATP 
- tripalmitin
- Phosphate
- CoA

I need to identify each metabolite in the model to be able to express the reaction.
I will collect them in the *metabolites* list

In [7]:
metabolites = []

Metabolites are available in the *model.metabolites* property. This is a special data structure called DictList.
This behaves essentially as a list from the users point of view.

In [8]:
type(model.metabolites)

cobra.core.dictlist.DictList

I can search for metabolites by name using the **query** method.

In [9]:
for m in model.metabolites.query("Glycerol 3-phosphate", attribute='name'):
    print(m)

glyc3p_c
glyc3p_m


In [10]:
model.metabolites.glyc3p_m

Metabolite identifier,glyc3p_m
Name,Glycerol 3-phosphate
Memory address,0x07f3e5722aa90
Formula,C3H7O6P
Compartment,m
In 4 reaction(s),"CDPDGPm_SC, GLYC3Ptm, G3PDm, G3PD1irm"


In [11]:
type(model.metabolites.glyc3p_m)

cobra.core.metabolite.Metabolite

The glyc3p_m metabolite is probably mitochondrial glycerol 3 phosphate.

In [12]:
model.compartments

{'c': 'cytosol',
 'm': 'mitochondria',
 'e': 'extracellular space',
 'x': 'peroxisome/glyoxysome',
 'r': 'endoplasmic reticulum',
 'v': 'vacuole',
 'n': 'nucleus',
 'g': 'golgi apparatus'}

In [13]:
model.metabolites.glyc3p_c

Metabolite identifier,glyc3p_c
Name,Glycerol 3-phosphate
Memory address,0x07f3e5722a9b0
Formula,C3H7O6P
Compartment,c
In 6 reaction(s),"G3PT, G3PD1ir, GAT1_SC, GLYC3Ptm, GPDDA1, GLYK"


The glyc3p_c metabolite is cytosolic Glycerol 3-phosphate. This is the one I want.
Add this to the *metabolites* list.

In [14]:
metabolites.append(model.metabolites.glyc3p_c)

In [15]:
m=model.metabolites.glyc3p_c

The metabolites have a number of useful properties:

In [16]:
print( model.metabolites.glyc3p_c.charge )
print( model.metabolites.glyc3p_c.elements )
print( model.metabolites.glyc3p_c.formula )
print( model.metabolites.glyc3p_c.formula_weight )
print( model.metabolites.glyc3p_c.model )

-2
{'C': 3, 'H': 7, 'O': 6, 'P': 1}
C3H7O6P
170.057841
iMM904


In [17]:
for m in model.metabolites.query("Palmitoyl-CoA", attribute='name'):
    print(m)

pmtcoa_c
pmtcoa_x


In [18]:
model.metabolites.pmtcoa_c

Metabolite identifier,pmtcoa_c
Name,Palmitoyl-CoA (n-C16:0CoA)
Memory address,0x07f3e5692fb38
Formula,C37H62N7O17P3S
Compartment,c
In 16 reaction(s),"FACOAL160, FA160COAabcp, EPISTAT_SC, ZYMSTAT_SC, DESAT16, TRIGS_SC, LPCAT_SC, FAS180COA, AGAT_SC, FAS160COA, FECOSTAT_SC, GAT1_SC, ERGSTAT_SC, LANOSTAT_SC, GAT2_SC, SERPT"


The metabolite pmtcoa_c is cytosolic Palmitoyl-CoA

In [19]:
metabolites.append(model.metabolites.pmtcoa_c)

Is there perhaps already a reaction involving cytosolic Palmitoyl-CoA? I should find out as this might affect the results.
The metabolite has a proterty called *reactions* which list all reactions where the metabolite takes part.

In [20]:
model.metabolites.pmtcoa_c.reactions

frozenset({<Reaction AGAT_SC at 0x7f3e56a5aa20>,
           <Reaction DESAT16 at 0x7f3e56aa3940>,
           <Reaction EPISTAT_SC at 0x7f3e56a894a8>,
           <Reaction ERGSTAT_SC at 0x7f3e56a896d8>,
           <Reaction FA160COAabcp at 0x7f3e56ad1860>,
           <Reaction FACOAL160 at 0x7f3e56ac8828>,
           <Reaction FAS160COA at 0x7f3e56abea20>,
           <Reaction FAS180COA at 0x7f3e56abe1d0>,
           <Reaction FECOSTAT_SC at 0x7f3e56b38eb8>,
           <Reaction GAT1_SC at 0x7f3e56b27eb8>,
           <Reaction GAT2_SC at 0x7f3e56b27f60>,
           <Reaction LANOSTAT_SC at 0x7f3e56c27748>,
           <Reaction LPCAT_SC at 0x7f3e56c275c0>,
           <Reaction SERPT at 0x7f3e56cb4b70>,
           <Reaction TRIGS_SC at 0x7f3e56ca4588>,
           <Reaction ZYMSTAT_SC at 0x7f3e56c894a8>})

In [21]:
type(model.metabolites.pmtcoa_c.reactions)

frozenset

[Frozenset](https://docs.python.org/3/library/stdtypes.html#frozenset) is a set that cannot be changed after its creation. The elements are *reactions*.
Lets have a look at the reaction type.

In [22]:
for r in model.metabolites.pmtcoa_c.reactions:
    break

In [23]:
type(r)

cobra.core.reaction.Reaction

The Reaction type has several useful methods and properties as well.
The check_mass_balance method is useful for newly defined reactions.

In [24]:
print( r.name )
print( r.check_mass_balance() )
print( r.gene_name_reaction_rule )
print( r.gene_reaction_rule )
print( r.build_reaction_string() )

Fatty acid  CoA ligase  hexadecanoate 
{}
FAA1 or FAA3 or FAA4
YOR317W or YIL009W or YMR246W
atp_c + coa_c + hdca_c <=> amp_c + pmtcoa_c + ppi_c


In [25]:
for reaction in model.metabolites.pmtcoa_c.reactions:
    print(f"{reaction.id:<15} {reaction.name}")

FACOAL160       Fatty acid  CoA ligase  hexadecanoate 
FA160COAabcp    Fatty acyl CoA peroxisomal transport via ABC system
EPISTAT_SC      Acyl CoAepisterol acyltransferase  yeast specific
ZYMSTAT_SC      Acyl CoAzymosterol acyltransferase  yeast specific
DESAT16         Palmitoyl CoA desaturase  n C160CoA   n C161CoA 
TRIGS_SC        Triglycerol synthesis
LPCAT_SC        Lyso phosphatidylcholine acyltransferase acyltransferase  yeast specific
FAS180COA       Fatty acyl CoA synthase  n C180CoA 
AGAT_SC         1 Acyl glycerol 3 phosphate acyltransferase  yeast specific
FAS160COA       Fatty acyl CoA synthase  n C160CoA 
FECOSTAT_SC     Acyl CoAfecosterol acyltransferase  yeast specific
GAT1_SC         Glycerol 3 phosphate acyltransferase  glycerol 3 phosphate   yeast specific
ERGSTAT_SC      Acyl CoAergosterol acyltransferase  yeast specific
LANOSTAT_SC     Acyl CoAlanosterol acyltransferase  yeast specific
GAT2_SC         Glycerol 3 phosphate acyltransferase  glycerone phosphate   yea

There is a reaction called *Triglycerol synthesis* above.

In [26]:
model.reactions.TRIGS_SC

Reaction identifier,TRIGS_SC
Name,Triglycerol synthesis
Memory address,0x07f3e56ca4588
Stoichiometry,0.01 12dgr_SC_c + 0.02 dcacoa_c + 0.06 ddcacoa_c + 0.17 hdcoa_c + 0.09 ocdycacoa_c + 0.24 odecoa_c + 0.27 pmtcoa_c + 0.05 stcoa_c + 0.1 tdcoa_c --> coa_c + 0.01 triglyc_SC_c 0.01 1 2 Diacylglycerol yeast specific C3540H6644O500 + 0.02 Decanoyl-CoA (n-C10:0CoA) + 0.06 Dodecanoyl-CoA (n-C12:0CoA) + 0.17 Hexadecenoyl-CoA (n-C16:1CoA) + 0.09 Octadecynoyl CoA n C182CoA C...
GPR,YOR245C or YCR048W or YNR019W
Lower bound,0.0
Upper bound,999999.0


In [27]:
model.reactions.TRIGS_SC.build_reaction_string()

'0.01 12dgr_SC_c + 0.02 dcacoa_c + 0.06 ddcacoa_c + 0.17 hdcoa_c + 0.09 ocdycacoa_c + 0.24 odecoa_c + 0.27 pmtcoa_c + 0.05 stcoa_c + 0.1 tdcoa_c --> coa_c + 0.01 triglyc_SC_c'

This reaction seem to produce a mixture of triacyl glycerols with fatty acids from 10 - 18 carbons.

In [28]:
for m in model.reactions.TRIGS_SC.metabolites:
     print(f"{m.id:<15} {m.name}")

12dgr_SC_c      1 2 Diacylglycerol  yeast specific C3540H6644O500
coa_c           Coenzyme A
dcacoa_c        Decanoyl-CoA (n-C10:0CoA)
ddcacoa_c       Dodecanoyl-CoA (n-C12:0CoA)
hdcoa_c         Hexadecenoyl-CoA (n-C16:1CoA)
ocdycacoa_c     Octadecynoyl CoA  n C182CoA  C39H62N7O17P3S
odecoa_c        Octadecenoyl-CoA (n-C18:1CoA)
pmtcoa_c        Palmitoyl-CoA (n-C16:0CoA)
stcoa_c         Stearoyl-CoA (n-C18:0CoA)
tdcoa_c         Tetradecanoyl-CoA (n-C14:0CoA)
triglyc_SC_c    Triglyceride  yeast specific C5160H9566O600


In [29]:
print( model.reactions.TRIGS_SC.gene_reaction_rule )

YOR245C or YCR048W or YNR019W


This reaction seem to be a general lipid reaction for S. cerevisiae. This is most likely there to provide triacylglycerol for biomass as indicated by the mixture of different fatty acid chain lengths.

[YOR245C](https://www.yeastgenome.org/locus/S000005771) is Diacylglycerol acyltransferase which catalyze the last step of triacylglycerol (TAG) formation.

Maybe the name of cytosolic atp is atp_c by convention?

In [30]:
model.metabolites.atp_c

Metabolite identifier,atp_c
Name,ATP C10H12N5O13P3
Memory address,0x07fb8896d1f60
Formula,C10H12N5O13P3
Compartment,c
In 149 reaction(s),"FTHFCL, TYRTRS, GSNK, ATPPRT, CBPS, PIN4K_SC, THFGLUS, DHFS, RNMK, FACOAL140, RNTR1, PI4P5K_SC, PIN3K_SC, TRPTRS, CTPS1, PI3P4K_SC, CTPS2, PI3P5K_SC, PRPPS, FTHFI, ATPtp_H, PANTS, PROTRS, RBK, FTHF..."


Metabolite atp_c is cytosolic ATP

In [31]:
metabolites.append(model.metabolites.atp_c)

In [32]:
metabolites

[<Metabolite glyc3p_c at 0x7fb8896094e0>,
 <Metabolite pmtcoa_c at 0x7fb888c93978>,
 <Metabolite atp_c at 0x7fb8896d1f60>]

Phosphate is produced and there seem to be many kinds of phosphate associated to different compartments.

In [33]:
model.metabolites.query("Phosphate", attribute='name')

[<Metabolite pi_c at 0x7fb888c93a58>,
 <Metabolite pi_e at 0x7fb888c93860>,
 <Metabolite pi_g at 0x7fb888c93b70>,
 <Metabolite pi_m at 0x7fb888c93a90>,
 <Metabolite pi_n at 0x7fb888c93ac8>,
 <Metabolite pi_r at 0x7fb888c93a20>,
 <Metabolite pi_v at 0x7fb888c937f0>,
 <Metabolite pi_x at 0x7fb888c939e8>]

In [34]:
model.metabolites.pi_c

Metabolite identifier,pi_c
Name,Phosphate
Memory address,0x07fb888c93a58
Formula,HO4P
Compartment,c
In 109 reaction(s),"GLNS, FTHFCL, PIt2p, PIt2n, FBP26, CBPS, CHORS, ASAD, THFGLUS, DHFS, LPP_SC, GTPH1, PIt2m, PIt2r, PIt2v, PSP_L, PIt5m, FBP, GLCP, NDP3, MGSA, NADPPPS, CTPS1, AKP1, DBTS, CTPS2, DKMPPD2, MI1PP, PI35..."


Metabolite pi_c is cytosolic ortophosphate 

In [35]:
metabolites.append(model.metabolites.pi_c)

by convention adp_c should be cytosolic adp

In [36]:
model.metabolites.adp_c

Metabolite identifier,adp_c
Name,ADP C10H12N5O10P2
Memory address,0x07fb88972a748
Formula,C10H12N5O10P2
Compartment,c
In 107 reaction(s),"GLNS, FTHFCL, PAK_SC, GSNK, NDPK8, NDPK9, GALKr, CBPS, NDPK7, PIN4K_SC, NDPK5, THFGLUS, NDPK3, DHFS, NDPK4, RNMK, HMPK1, PI4P5K_SC, PIN3K_SC, NDPK6, CHOLK, CTPS1, PI3P4K_SC, DBTS, CTPS2, PI3P5K_SC,..."


Metabolite adp_c is cytosolic adp.

In [37]:
metabolites.append(model.metabolites.adp_c)

by convention coa_c should be cytosolic coenzyme A (CoA)

In [38]:
model.metabolites.coa_c

Metabolite identifier,coa_c
Name,Coenzyme A
Memory address,0x07fb8896f6240
Formula,C21H32N7O16P3S
Compartment,c
In 64 reaction(s),"ACOATA, FAS80_L, CSNATr, FACOAL140, ACGAM6PS, FAS100COA, GLYAT, FAS120, FAS180COA, DIAT, FAS100, FACOAL181, FAS180, FAS181, LPCAT_SC, TRIGS_SC, OHACT5, ZYMSTAT_SC, AGAT_SC, LANOSTAT_SC, OHACT2, MAL..."


Metabolite coa_c is cytosolic CoA.

In [39]:
metabolites.append(model.metabolites.coa_c)

The metabolites list is now complete except for tripalmitin. I know that tripalmitin has the elemental formula C51H98O6.
I would have expected that the following would be allowed.

    model.metabolites.query("C51H98O6", attribute="formula")

but this gives me an error like this

    ---------------------------------------------------------------------------
    TypeError                                 Traceback (most recent call last)
    <ipython-input-103-d3022279d4ab> in <module>
    ----> 1 model.metabolites.query("C51H98O6", attribute="formula")

    ~/anaconda3/envs/cameo/lib/python3.6/site-packages/cobra/core/dictlist.py in query(self, search_function, attribute)
        148 
        149         results = self.__class__()
    --> 150         results._extend_nocheck(matches)
        151         return results
        152 

    ~/anaconda3/envs/cameo/lib/python3.6/site-packages/cobra/core/dictlist.py in _extend_nocheck(self, iterable)
        209         """
        210         current_length = len(self)
    --> 211         list.extend(self, iterable)
        212         _dict = self._dict
        213         if current_length is 0:

    ~/anaconda3/envs/cameo/lib/python3.6/site-packages/cobra/core/dictlist.py in <genexpr>(.0)
        135                 matches = (
        136                     i for i in self if
    --> 137                     regex_searcher.findall(select_attribute(i)) != [])
        138 
        139             else:

    TypeError: expected string or bytes-like object

The following is allowed, where I search for name or id: 

In [40]:
model.metabolites.query("H2O", attribute="name")

[<Metabolite h2o_c at 0x7fb8895b8da0>,
 <Metabolite h2o_e at 0x7fb8895b8e48>,
 <Metabolite h2o_g at 0x7fb8895b8eb8>,
 <Metabolite h2o_m at 0x7fb8895b8f28>,
 <Metabolite h2o_n at 0x7fb8895b8f98>,
 <Metabolite h2o_r at 0x7fb8895c17f0>,
 <Metabolite h2o_v at 0x7fb8895c1be0>,
 <Metabolite h2o_x at 0x7fb8895c1ba8>]

In [41]:
model.metabolites.query("h2o_c", attribute="id")

[<Metabolite h2o_c at 0x7fb8895b8da0>]

However, I can use a list comprehension like this:

In [47]:
[m for m in model.metabolites if m.formula=="H2O"]

[<Metabolite h2o_c at 0x7fb8895b8da0>,
 <Metabolite h2o_e at 0x7fb8895b8e48>,
 <Metabolite h2o_g at 0x7fb8895b8eb8>,
 <Metabolite h2o_m at 0x7fb8895b8f28>,
 <Metabolite h2o_n at 0x7fb8895b8f98>,
 <Metabolite h2o_r at 0x7fb8895c17f0>,
 <Metabolite h2o_v at 0x7fb8895c1be0>,
 <Metabolite h2o_x at 0x7fb8895c1ba8>]

There is only one metabolite with C51 in the formula as would be expected of tripalmitin.
This is the one linked to biomass.

In [57]:
[m for m in model.metabolites if m.formula and "C51" in m.formula]

[<Metabolite triglyc_SC_c at 0x7fb888ce2f98>]

In [58]:
[m for m in model.metabolites if m.formula=="C51H98O6"]

[]

In [60]:
model.metabolites.triglyc_SC_c

Metabolite identifier,triglyc_SC_c
Name,Triglyceride yeast specific C5160H9566O600
Memory address,0x07fb888ce2f98
Formula,C5160H9566O600
Compartment,c
In 4 reaction(s),"BIOMASS_SC5_notrace, TRIGS_SC, PCDAGAT, TAGL_SC"


I have got all metabolites in the *metabolites* list except tripalmitin.

In [61]:
for metabolite in metabolites:
    print(f"{metabolite.name:<30} {metabolite}")

Glycerol 3-phosphate           glyc3p_c
Palmitoyl-CoA (n-C16:0CoA)     pmtcoa_c
ATP C10H12N5O13P3              atp_c
Phosphate                      pi_c
ADP C10H12N5O10P2              adp_c
Coenzyme A                     coa_c


I have to create the tripalmitin metabolite as it is not part of the model.

In [28]:
from cobra import Metabolite

In [29]:
tripmt_c = Metabolite(id="tripmt_c", name = "Tripalmitin" , formula="C51H98O6", compartment="c")

The name tripmt_c follows the convention roughly. I got the formula from [wikipedia](https://en.wikipedia.org/wiki/Tripalmitin).

In [30]:
tripmt_c

Metabolite identifier,tripmt_c
Name,Tripalmitin
Memory address,0x07f3e538b3128
Formula,C51H98O6
Compartment,c
In 0 reaction(s),


Now I have to create a new reaction and make it a part of the model.

This is the reaction as stated before:

    Glycerol 3-phosphate + 3 Palmitoyl-CoA + 2 ATP        -> tripalmitin + 2 ADP + 3 CoA + 3 Phosphate    

In [31]:
from cobra import Reaction

The reaction can be created by a dict where the metabolites are keys and the stoichiometric coefficient
are values. A negative value indicate a substrate and a positive one product.

In [32]:
reaction = Reaction('tripalmitin_synth')
reaction.name = 'synthesis of tripalmitin'
reaction.add_metabolites({
    model.metabolites.glyc3p_c: -1.0,
    model.metabolites.pmtcoa_c: -3.0,
    model.metabolites.atp_c   : -2.0,
    tripmt_c                  :  1.0,
    model.metabolites.pi_c    :  3.0,
    model.metabolites.adp_c   :  2.0,
    model.metabolites.coa_c   :  3.0,
})
reaction.reaction

'2.0 atp_c + glyc3p_c + 3.0 pmtcoa_c --> 2.0 adp_c + 3.0 coa_c + 3.0 pi_c + tripmt_c'

The reaction comes with a *check_mass_balance* method. 

In [33]:
reaction.check_mass_balance()

{'charge': -2.0, 'H': 4.0, 'O': 3.0}

Apparently charge, oxygen and hydrogen are not balanced. I probably forgot to add
protons and water to the equation.

h_c is cytosolic H+

In [34]:
model.metabolites.h_c

Metabolite identifier,h_c
Name,H+
Memory address,0x07f3e571e3fd0
Formula,H
Compartment,c
In 539 reaction(s),"FTHFI, FAS100, D_LACt2, CSND, DRBK, FMNAT, DROPPRy, DURIK1, FRUK, FRUt2, FMNRx, DURIt2, FMNRx2, DUTPDP, CPPPGO, DRTPPD, FALDH, DPR, DOLK, FKYNH, DPGM, HEX1, FDNG, DPCOAK, ETHAPT_SC, DOLPt2er, FECOS..."


h2o_c is cytosolic water 

In [35]:
model.metabolites.h2o_c

Metabolite identifier,h2o_c
Name,H2O H2O
Memory address,0x07f3e571d68d0
Formula,H2O
Compartment,c
In 274 reaction(s),"FTHFI, PSPHS, FAS100, PSP_L, TAGL_SC, CSND, TREH, SULR, DXHPScm, SSALy, PLD_SC, TRPS1, PRAMPC, TRE6PP, NTD3, DUTPDP, CPPPGO, HCO3E, PRATPP, PRFGS, NTP3, DRTPPD, H2Otv, TMN, NTRLASE, NTRLASE2, SHSL4..."


I ended up adding water and protons by trial and error until the reaction is balanced for both charge and elements.

In [36]:
reaction = Reaction('tripalmitin_synth')
reaction.name = 'synthesis of tripalmitin'
reaction.add_metabolites({
    model.metabolites.glyc3p_c: -1.0,
    model.metabolites.pmtcoa_c: -3.0,
    model.metabolites.atp_c   : -2.0,
    model.metabolites.h2o_c   : -3.0,
    tripmt_c                  :  1.0,
    model.metabolites.pi_c    :  3.0,
    model.metabolites.adp_c   :  2.0,
    model.metabolites.coa_c   :  3.0,
    model.metabolites.h_c     :  2.0,
})
reaction.reaction

'2.0 atp_c + glyc3p_c + 3.0 h2o_c + 3.0 pmtcoa_c --> 2.0 adp_c + 3.0 coa_c + 2.0 h_c + 3.0 pi_c + tripmt_c'

In [37]:
reaction.check_mass_balance()

{}

An empty dict from check_mass_balance indicate a balanced reaction as showed [here](https://cobrapy.readthedocs.io/en/latest/getting_started.html).

I have been adviced to also create a demand reaction for export of the product. This can be thought of as a specific exporter of the 
product to the extracellular environment.

In [38]:
export_reaction = Reaction('tripalmitin_exp')
export_reaction.name = 'export of tripalmitin'
export_reaction.add_metabolites({ tripmt_c : -1.0 })
export_reaction.reaction

'tripmt_c --> '

The method add_reaction can be used to add a list of reactions to the model.

In [39]:
model.add_reactions([reaction, export_reaction])

Now we must set an objective for the model.

In [40]:
model.objective = 'tripalmitin_exp'

In [41]:
model.objective.expression

1.0*tripalmitin_exp - 1.0*tripalmitin_exp_reverse_63d5c

There are several built in methods that can be used to evaluate the model and objective.

The optimize method and the pfba function use slightly different methods.  

In [42]:
fba_solution = model.optimize()
pfba_solution = cobra.flux_analysis.pfba(model)

In [43]:
model.summary()

IN FLUXES       OUT FLUXES    OBJECTIVES
--------------  ------------  ----------------------
o2_e      12.7  h2o_e  28.1   tripalmitin_exp  0.652
glc__D_e  10    co2_e  26.7


In [44]:
fba_solution.status, pfba_solution.status

('optimal', 'optimal')

In [45]:
fba_solution.fluxes.EX_glc__D_e, fba_solution.fluxes.tripalmitin_exp

(-10.0, 0.652032972121886)

In [46]:
pfba_solution.fluxes.EX_glc__D_e, pfba_solution.fluxes.tripalmitin_exp

(-10.0, 0.652032972121886)

The two methods give identical results in this case.

In [47]:
molar_yeld = -1. * fba_solution.fluxes.tripalmitin_exp / fba_solution.fluxes.EX_glc__D_e
molar_yeld

0.06520329721218861

In [48]:
mw_glc = model.metabolites.glc__D_c.formula_weight
mw_trip = model.metabolites.tripmt_c.formula_weight
mw_trip, mw_glc

(807.3202200000001, 180.15588)

In [49]:
print( molar_yeld  * (mw_trip/mw_glc), "g tripalmitin / g glucose")

0.2921910750294106 g tripalmitin / g glucose


### Results

The results was 0.0652 mol tripalmitin / mol glucose  or 0.292 g/g tripalmitin per g glucose

Is this a reasonable yield? The publication [Koivuranta et al. 2018](https://www.frontiersin.org/articles/10.3389/fmicb.2018.01337/full) describe
metabolic modelling of Trichosporon oleaginosus (Cryptococcus curvatus) and claim a theoretical yield of 0.289 g/g. This is very close to 
the results here. A genome scale model for _Yarrowia lipolytica_ was used. The reference for the model in the paper is [Kerkhoven et al. 2016](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5516929/) and the model was named iYali4.

The model had a limit for flux to the oxidative pentose phosphate pathway to 52.0% of glucose uptake among other constrainsts. The conversion of molar yields to g/g was done by assuming the FA composition of acyl-CoA in the model1, giving a molar mass of 826.678 g/mol for TAG.


## Manual estimate of yield

Below is an attempt to manually calculating yield. 
Assumptions are Glycerol 3 phosphate from glycolysis (B)
NADPH from oxidative pentose phosphate pathway (E). 
No interconversion between NADH and NADPH. 

    A) 1 Glycerol 3-phosphate + 3 Palmitoyl-CoA + 2 ATP --> 1 tripalmitin

    B) 1 Glucose + 2 ATP + 2 NADH                       --> 2 Glycerol 3-phosphate

    C) 8 Acetyl-CoA + 7 ATP + 14 NADPH                  --> 1 Palmitoyl-CoA

    D) 1 Glucose                                        --> 2 ATP + 4 NADH + 2 Acetyl-CoA

    E) 1 Glucose                                        --> 12 NADPH

#### Glucose ⟶ Glycerol 3-phosphate 

    A)   1 Glycerol 3-phosphate + 3 Palmitoyl-CoA + 2 ATP   --> 1 tripalmitin
    B)   1 Glucose + 2 ATP + 2 NADH                         --> 2 Glycerol 3-phosphate
    B) 0.5 Glucose + 1 ATP + 1 NADH                         --> 1 Glycerol 3-phosphate


F combines A and B to eliminate Glycerol 3-phosphate.

    F) 0.5 Glucose + 3 ATP + 1 NADH + 3 Palmitoyl-CoA    --> 1 tripalmitin 

#### Acetyl-CoA ⟶ Palmitoyl-CoA 

    F)  0.5 Glucose + 3 ATP + 1 NADH + 3 Palmitoyl-CoA   --> 1 tripalmitin
    C)   8 Acetyl-CoA +  7 ATP + 14 NADPH                --> 1 Palmitoyl-CoA
    C)  24 Acetyl-CoA + 21 ATP + 42 NADPH                --> 3 Palmitoyl-CoA
    
G is a combination of F and C.

    G) 0.5 Glucose + 1 NADH + 24 Acetyl-CoA + 24 ATP + 42 NADPH --> 1 tripalmitin

#### Glucose ⟶ Acetyl-CoA

    G) 0.5 Glucose + 1 NADH + 24 Acetyl-CoA + 24 ATP + 42 NADPH --> 1 tripalmitin

    D)  1 Glucose + 2 ADP  -> 2 ATP + 4 NADH + 2 Acetyl-CoA
    D) 12 Glucose + 24 ADP  -> 24 ATP + 48 NADH + 24 Acetyl-CoA

H is a combination of G and D.

    H) 12.5 Glucose + 42 NADPH   --> 1 tripalmitin + 47 NADH

#### Glucose ⟶ NADPH

    H) 12.5 Glucose + 42 NADPH   --> 1 tripalmitin + 47 NADH
    E)   1 Glucose               --> 12 NADPH
    E) 3.5 Glucose               --> 42 NADPH

    I) 16 Glucose                --> 1 tripalmitin + 47 NADH
    
There is a surplus of NADH in (I) which can be used for ATP production. If it could be turned into 
NADPH, the 3.5 glucose used in PPP would be saved.

Thus the yield is 1/16 = 0.0625 mol/mol which gives

(807.3202200000001 / 180.15588) * 0.0625 = 0.280 g/g

This is a lower yield than the one calculated by the model but only by about 5%

[Escher](https://escher.github.io/) is a software to build, share, and embed visualizations of biological pathways.

In [53]:
import escher

In [54]:
reaction_data = fba_solution.fluxes[fba_solution.fluxes.abs() > 1E-07].to_dict()

In [55]:
escher.Builder(map_name='iMM904.Central carbon metabolism',
               model_name='iMM904',
               reaction_data=reaction_data).display_in_notebook()

There are some reactions with nd instead of flux numbers in the model above.